In [1]:
#!pip install timm

In [2]:
import pandas as pd
import timm
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
#from efficientnet_pytorch import EfficientNet
import numpy as np

from PIL import Image
from PIL import ImageFile

In [3]:
base_dir = r'C:\Users\Kaggle\Leaf_Classification\input'

In [4]:
### read csv and label mapping

labels = pd.read_csv(os.path.join(base_dir,'train.csv'))

with open(f'{base_dir}/label_num_to_disease_map.json') as f:
    label_mapper = json.load(f)

### Dataset class

In [5]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

### Augs

In [6]:
sz = 576
sz_efffnetb4 = 600

In [7]:
infer_augs_effnet = albumentations.Compose([
            albumentations.RandomResizedCrop(sz_efffnetb4, sz_efffnetb4),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)


infer_augs_resnext = albumentations.Compose([
            albumentations.RandomResizedCrop(sz, sz),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5), 
            ToTensorV2()],
            p=1.)

### PL module

In [8]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -1,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if source == 'timm':
            net = timm.create_model(arch, pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        else:
            print("Not implemented")
            
        if freeze:
            freeze_until(net, freeze_until_layer)
        
        layers = list(net.children())[:n_layers]    
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [9]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Linear(output_dimension , n_classes)

    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [10]:
class Net_B4(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b4_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [11]:
class Net_B5(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='tf_efficientnet_b5_ns',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [12]:
class BasicDecoder_seresnext(nn.Module):
    def __init__(self, pool_output_size: int = 1, n_classes: int = 5, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        #self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        #self.fc = nn.Linear(output_dimension , n_classes)
        self.fc = nn.Sequential(
            nn.Linear(output_dimension, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(512, 5),
        )
    def forward(self, x):
        #x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))
        return output

In [13]:
class seResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='seresnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder_seresnext(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [14]:
class ResnextNet(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnext50_32x4d',
                                    source='timm',
                                    pretrained=False,
                                    n_layers=-1,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(1,5,self.encoder.output_dimension)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss

In [15]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 1e-4

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

In [16]:
labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [17]:
# path = "../input/cassava-leaf-disease-classification/"
# sub = pd.read_csv(f'{path}/sample_submission.csv')
# sub.head()

In [18]:
def getloader(df,train_augs,valid_augs,path,bs=8,fold=0):

    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    train_indexes, valid_indexes = list(folds.split(df, df['label']))[fold]

    train_df = df.iloc[train_indexes]
    valid_df = df.iloc[valid_indexes]


    train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                    transforms=train_augs,
                                                    labels=train_df['label'].values,
                                                    img_path=path,
                                                    mode='train',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                    transforms=valid_augs,
                                                    labels=valid_df['label'].values,
                                                    img_path=path,
                                                    mode='valid',
                                                    labels_to_ohe=False,
                                                    n_classes=5)
    
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=bs,num_workers=0,shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=bs,num_workers=0,shuffle=False)

    return train_loader,valid_loader,valid_indexes

In [19]:
# models = os.listdir('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss/')
# modelsPath = [os.path.join('../input/leaves-tf-enet-b4-ns-576-finetune-temperedloss',i) for i in models]
# modelsPath

In [20]:
from tqdm import tqdm
def getPreds(test_loader,lit_model,logitsOut=True):
    predictions = []
    
    for batch in tqdm(test_loader):
        image = batch['image'].to('cuda')
        target = batch['target'].to('cuda')
        with torch.no_grad():
            outputs = lit_model.model(image, target)[0]
            _outputs = outputs.softmax(1).detach().cpu().numpy()
            preds = outputs.argmax(1).detach().cpu().numpy()
            if not logitsOut:
                predictions.append(preds)
            else:
                #predictions[range(len(_outputs))] += _outputs
                predictions.append(_outputs)
    return np.array(predictions)

### Preds for seresnext

In [30]:
#modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\'')

modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\0\epoch=11_valid_loss=0.1303_valid_accuracy=0.8974.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\1\epoch=12_valid_loss=0.1256_valid_accuracy=0.9012.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\2\epoch=11_valid_loss=0.1299_valid_accuracy=0.8997.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\3\epoch=15_valid_loss=0.1374_valid_accuracy=0.8890.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\4\epoch=8_valid_loss=0.1392_valid_accuracy=0.8862.ckpt'}
             


In [22]:
# modelsDir = os.listdir(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\)

# modelsPath = [os.path.join(r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\seresnext50_32x4d_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\best_ones',i) for i in models]
# modelsPath

In [31]:
preds_seresnext = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    _,test_loader,valid_ix = getloader(labels,infer_augs_resnext,infer_augs_resnext,path=train_img_path,bs=128,fold=f)
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = seResnextNet()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_seresnext[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


100%|██████████| 34/34 [01:58<00:00,  3.48s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 34/34 [02:00<00:00,  3.54s/it]


Prediction for fold:1


100%|██████████| 34/34 [01:59<00:00,  3.50s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 34/34 [01:58<00:00,  3.50s/it]


Prediction for fold:2


100%|██████████| 34/34 [01:59<00:00,  3.53s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 34/34 [01:58<00:00,  3.48s/it]


Prediction for fold:3


100%|██████████| 34/34 [01:57<00:00,  3.47s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 34/34 [01:57<00:00,  3.47s/it]


Prediction for fold:4


100%|██████████| 34/34 [01:57<00:00,  3.46s/it]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 34/34 [01:57<00:00,  3.46s/it]


In [36]:
preds_mean = preds_seresnext.copy()
preds_mean[:,0]= preds_seresnext[:,0]/(1+np.sqrt(0.05))
preds_mean[:,1]= preds_seresnext[:,1]/(1+np.sqrt(0.102))
preds_mean[:,2]= preds_seresnext[:,2]/(1+np.sqrt(0.111))
preds_mean[:,3]= preds_seresnext[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean[:,4]= preds_seresnext[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean,1)

In [33]:
# classOut = np.argmax(preds_seresnext,1)
# classOut.shape

(21397,)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_seresnext,1))

(0.8951722204047297, 0.8958265177361312)

In [38]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect()

388

In [39]:
torch.cuda.empty_cache()

### Preds for B4

In [23]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\0\epoch=4_valid_loss=0.0972_valid_accuracy=0.8949.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\1\epoch=4_valid_loss=0.0919_valid_accuracy=0.9012.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\2\epoch=8_valid_loss=0.0967_valid_accuracy=0.8923.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\3\epoch=8_valid_loss=0.1017_valid_accuracy=0.8885.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\4\epoch=0_valid_loss=0.1005_valid_accuracy=0.8913.ckpt'
             }
             


In [24]:
preds_effnet_b4 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    _,test_loader,valid_ix = getloader(labels,infer_augs_effnet,infer_augs_effnet,path=train_img_path,bs=16,fold=f)
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B4()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b4[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


100%|██████████| 268/268 [02:00<00:00,  2.22it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [01:56<00:00,  2.31it/s]


Prediction for fold:1


100%|██████████| 268/268 [01:55<00:00,  2.32it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [01:55<00:00,  2.31it/s]


Prediction for fold:2


100%|██████████| 268/268 [01:56<00:00,  2.31it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [01:56<00:00,  2.31it/s]


Prediction for fold:3


100%|██████████| 268/268 [01:56<00:00,  2.30it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [01:56<00:00,  2.31it/s]


Prediction for fold:4


100%|██████████| 268/268 [01:57<00:00,  2.29it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [01:56<00:00,  2.30it/s]


In [25]:
preds_mean_effnetb4 = preds_effnet_b4.copy()
preds_mean_effnetb4[:,0]= preds_effnet_b4[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb4[:,1]= preds_effnet_b4[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb4[:,2]= preds_effnet_b4[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb4[:,3]= preds_effnet_b4[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb4[:,4]= preds_effnet_b4[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb4,1)

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b4,1))

(0.8959199887834743, 0.8970416413515914)

In [28]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Preds for B5

In [49]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\0\epoch=0_valid_loss=0.0980_valid_accuracy=0.8949.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\1\epoch=4_valid_loss=0.0926_valid_accuracy=0.9021.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\2\epoch=7_valid_loss=0.0942_valid_accuracy=0.8986.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\3\epoch=4_valid_loss=0.1031_valid_accuracy=0.8883.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\4\epoch=3_valid_loss=0.1046_valid_accuracy=0.8890.ckpt'
             }
             


In [50]:
preds_effnet_b5 = np.zeros((len(labels),5))

train_img_path = 'C:/Users/Kaggle/Leaf_Classification/input/train_images/'
TTA = 5
for f in range(5):
    _,test_loader,valid_ix = getloader(labels,infer_augs_effnet,infer_augs_effnet,path=train_img_path,bs=16,fold=f)
    softmaxOut = 0.
    print(f"Prediction for fold:{f}")
    modelPath = modelsPath[f]
    model = Net_B5()
    lit_model = LitCassava(model)
    checkpoint = torch.load(modelPath, map_location=lambda storage, loc: storage)
    lit_model.load_state_dict(checkpoint['state_dict'])  
    lit_model.model.eval()
    lit_model.model.cuda()
    for tta in range(TTA):
        predictions = []
        softmaxOut+= (getPreds(test_loader,lit_model,logitsOut=True))
    preds_effnet_b5[valid_ix,:] = np.vstack(softmaxOut)

Prediction for fold:0


100%|██████████| 268/268 [02:07<00:00,  2.10it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [02:08<00:00,  2.09it/s]


Prediction for fold:1


100%|██████████| 268/268 [02:07<00:00,  2.10it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [02:07<00:00,  2.10it/s]


Prediction for fold:2


100%|██████████| 268/268 [02:07<00:00,  2.10it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [02:08<00:00,  2.09it/s]


Prediction for fold:3


100%|██████████| 268/268 [02:07<00:00,  2.10it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [02:07<00:00,  2.10it/s]


Prediction for fold:4


100%|██████████| 268/268 [02:07<00:00,  2.10it/s]
C:\Users\Anaconda_Stuff\envs\Pytorch_17_cuda11\lib\site-packages\ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 268/268 [02:07<00:00,  2.10it/s]


In [51]:
preds_mean_effnetb5 = preds_effnet_b5.copy()
preds_mean_effnetb5[:,0]= preds_effnet_b5[:,0]/(1+np.sqrt(0.05))
preds_mean_effnetb5[:,1]= preds_effnet_b5[:,1]/(1+np.sqrt(0.102))
preds_mean_effnetb5[:,2]= preds_effnet_b5[:,2]/(1+np.sqrt(0.111))
preds_mean_effnetb5[:,3]= preds_effnet_b5[:,3]/(1+np.sqrt(0.614)) #### not the exact frequency stats here, instead of 0.614, we have used 0.4 
preds_mean_effnetb5[:,4]= preds_effnet_b5[:,4]/(1+np.sqrt(0.12))

classOut = np.argmax(preds_mean_effnetb5,1)

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(labels.label,classOut),accuracy_score(labels.label,np.argmax(preds_effnet_b5,1))

(0.8970883768752629, 0.8980230873486937)

In [53]:
del _,softmaxOut,model,lit_model,checkpoint,test_loader; gc.collect();torch.cuda.empty_cache()

### Effnet B6 - TBD

In [ ]:
modelsPath = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\0\epoch=0_valid_loss=0.0980_valid_accuracy=0.8949.ckpt',
              1:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\1\epoch=4_valid_loss=0.0926_valid_accuracy=0.9021.ckpt',
              2:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\2\epoch=7_valid_loss=0.0942_valid_accuracy=0.8986.ckpt',
              3:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\3\epoch=4_valid_loss=0.1031_valid_accuracy=0.8883.ckpt',
              4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b5_ns_sz_576_timm_finetune_TemperedLoss_10Epochs\4\epoch=3_valid_loss=0.1046_valid_accuracy=0.8890.ckpt'
             }
             


### Ensemble

In [67]:
i = 0.4
np.arange(0,1-i,0.01)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59])

In [70]:
res = pd.DataFrame()
res['b4_contrib'] = 0
res['b5_contrib'] = 0
res['seresnext_contrib'] = 0
res ['Accuracy'] = 0

count = 0
for i in np.arange(0,1,0.01):
    for j in np.arange(0,1-i,0.01):
        b4_contrib = i
        b5_contrib = j
        seresnext_contrib = 1-i-j
        #print(i,j,1-i-j)
        ens_preds = (b4_contrib*preds_effnet_b4+seresnext_contrib*preds_seresnext+b5_contrib*preds_effnet_b5)
        res.loc[count,'b4_contrib'] = i
        res.loc[count,'seresnext_contrib'] = 1-i-j
        res.loc[count,'b5_contrib'] = j
        res.loc[count,'Accuracy'] = accuracy_score(labels.label,np.argmax(ens_preds,1))
        count += 1

In [73]:
res.sort_values(by=['Accuracy'],ascending=False)

,b4_contrib,b5_contrib,seresnext_contrib,Accuracy
2874,0.34,0.35,0.31,0.902930
2807,0.33,0.35,0.32,0.902930
2804,0.33,0.32,0.35,0.902884
2875,0.34,0.36,0.30,0.902884
3009,0.36,0.39,0.25,0.902884
...,...,...,...,...
199,0.02,0.00,0.98,0.896714
101,0.01,0.01,0.98,0.896621
1,0.00,0.01,0.99,0.896481
100,0.01,0.00,0.99,0.896247
